# Exploring Source Data

The purpose of this notebook is to investigate a single source file to determine what is necessary for initialization and eventually, artifact rejection.

For instructive purposes, this notebook may be run in various orders to demonstrate how various functions behave.

The browser backend call below is a secondary type of import that allows for interactive and responsive figures.

In [1]:
import mne
mne.viz.set_browser_backend('qt') # Use the fancier figures when possible

Using qt as 2D backend.


In [2]:
subject_file = '../sourcedata/IC_trn_1.bdf'
raw = mne.io.read_raw(subject_file, preload=True) # Load the defined subject
raw # Last variable in a cell "spills" to the output

Extracting EDF parameters from /home/tyler/Documents/eeg-dev/StudyTemplate/sourcedata/IC_trn_1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 280575  =      0.000 ...   273.999 secs...


Measurement date,"August 31, 2009 14:25:27 GMT"
Experimenter,Unknown
Participant,
Digitized points,Not available
Good channels,"128 EEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,1024.00 Hz
Highpass,0.00 Hz
Lowpass,268.00 Hz


In [ ]:
raw = raw.filter(l_freq=1.0, h_freq=30.0)

In [ ]:
%matplotlib inline
# "magic" cell command, this one keeps the plots in the notebook instead of popping up
raw = raw.set_montage('biosemi128')
raw = raw.set_eeg_reference('average')
_ = raw.plot_sensors(show_names=True)

In [ ]:
%matplotlib qt
raw.plot_psd()

Plotting the scroll plot of the data can be useful at any time:

In [ ]:
%matplotlib qt
raw.plot()

### Exploring Epochs

There are two ways that MNE can extract events from files. Either via dedicated stim channels, or "annotations".

This example describes stim channels. As such, the `event_dict` must be manually input. These are your standard pin/markers.

In [3]:
# event_dict = {"boundary": 65791, "anim/house/upright/f1": 11, "anim/house/upright/f2": 12, "anim/house/upright/f3": 13, "anim/house/upright/f4": 14, "anim/house/upright/f5": 15, "anim/house/upright/f6": 16, "anim/face/upright/f1": 31, "anim/face/upright/f2": 32, "anim/face/upright/f3": 33, "anim/face/upright/f4": 34, "anim/face/upright/f5": 35, "anim/face/upright/f6": 36, "house/inverted/f1": 21, "anim/house/inverted/f2": 22, "anim/house/inverted/f3": 23, "anim/house/inverted/f4": 24, "anim/house/inverted/f5": 25, "anim/house/inverted/f6": 26, "anim/checker/f1": 51, "anim/checker/f2": 52, "anim/checker/f3": 53, "anim/checker/f4": 54, "anim/checker/f5": 55, "anim/checker/f6": 56, "anim/face/inverted/f1": 41, "anim/face/inverted/f2": 42, "anim/face/inverted/f3": 43, "anim/face/inverted/f4": 44, "anim/face/inverted/f5": 45, "anim/face/inverted/f6": 46, "static/checker/left": 215, "press/left": 201, "static/checker/right": 216, "press/right": 204, "static/face/upright": 211, "static/face/inverted": 212, "static/house/upright": 213, "static/house/inverted": 214}
event_dict = {
    "static/checker/left": 215,
    "press/left": 201,
    "static/checker/right": 216,
    "press/right": 204,
    "static/face/upright": 211,
    "static/face/inverted": 212,
    "static/house/upright": 213,
    "static/house/inverted": 214,
}
events = mne.find_events(raw)
events

Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
401 events found on stim channel Status
Event IDs: [  201   204   211   212   213   214   215   216 65790]


array([[   476,  65536,  65790],
       [  6401,      0,    215],
       [  6838,      0,    201],
       ...,
       [274806,      0,    215],
       [275169,      0,    201],
       [275624,      0,    211]])

The format of events is always an array with three elements.

1. The sample
2. The previously seen event (can be configured to help with segmentation)
3. The marker code

If you are interested in finding breaks in your task automatically you will need to switch to the "annotations" approach. Since this example is using a single Face13 subject, there are no annotations. Here's how to generate them programmatically using the previously defined information:

In [ ]:
anno = mne.annotations_from_events(events, raw.info['sfreq'], event_desc={v:k for k,v in event_dict.items()})
raw.set_annotations(anno)

In [ ]:
break_annots = mne.preprocessing.annotate_break(
    raw=raw,
    min_break_duration=5,
    t_start_after_previous=1,
    t_stop_before_next=1,
)

raw.set_annotations(raw.annotations + break_annots)  # add to existing

In [ ]:
raw.plot()

This function is complicated to understand. I strongly recommend visiting the documentation for it and exploring the effect of parameter choices by replotting you data and seeing where it places the breaks.

See documentation [here](https://mne.tools/stable/generated/mne.preprocessing.annotate_break.html#mne.preprocessing.annotate_break).

### Extremely Quick ICA Showcase

In [ ]:
# set up and fit the ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
ica.fit(raw)
ica.exclude = [1, 2] # details on how we picked these are omitted here
ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
raw = ica.apply(raw)
raw

### Finishing Epoching

If necessary, you can add channels to a list known as `bads`. In this case, MNE will always ignore it by default.

Be careful however, if you try and ask for it directly, it WILL be included in things. If you want to avoid this, you can interpolate via `interpolate_bads`.

In [ ]:
raw.info['bads'].append('C10')

In [ ]:
reject_criteria = dict(
    eeg=150e-6,  # 150 µV
)

epochs = mne.Epochs(
    raw,
    events,
    event_id=event_dict,
    tmin=-0.2,
    tmax=0.6,
    reject=reject_criteria,
    preload=True,
)
epochs

In [ ]:
%matplotlib inline
epochs['face'].average().plot_joint()
_ = epochs['face'].average().plot_topomap(times=[0.0, 0.08, 0.1, 0.12, 0.2])

In [ ]:
%matplotlib inline
comparison = {
    'press': epochs['press'].average(),
    'house': epochs['house'].average(),
}
mne.viz.plot_compare_evokeds(comparison, combine='mean', picks=['A22', 'A23'])

In [ ]:
epochs['press'].to_data_frame()